In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('PPP Data 150k plus 080820.csv.', parse_dates=['DateApproved'], infer_datetime_format=True)

In [ ]:
df.head()

In [6]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 662515 entries, 0 to 662514
Data columns (total 16 columns):
LoanRange        662515 non-null object
BusinessName     662510 non-null object
Address          662498 non-null object
City             662500 non-null object
State            662499 non-null object
Zip              662499 non-null float64
NAICSCode        655800 non-null float64
BusinessType     661110 non-null object
RaceEthnicity    662515 non-null object
Gender           662515 non-null object
Veteran          662515 non-null object
NonProfit        41993 non-null object
JobsReported     622635 non-null float64
DateApproved     662515 non-null datetime64[ns]
Lender           662515 non-null object
CD               662292 non-null object
dtypes: datetime64[ns](1), float64(3), object(12)
memory usage: 80.9+ MB


,Zip,NAICSCode,JobsReported
count,662499.000000,655800.000000,622635.000000
mean,51334.381745,504320.424614,50.324980
std,30763.300635,177773.534782,69.585824
min,256.000000,111110.000000,0.000000
25%,23606.000000,337110.000000,16.000000
50%,48726.000000,541110.000000,28.000000
75%,79701.000000,621320.000000,54.000000
max,99929.000000,999990.000000,500.000000


In [7]:
df['LoanRange'].value_counts()

e $150,000-350,000      380636
d $350,000-1 million    199679
c $1-2 million           53218
b $2-5 million           24248
a $5-10 million           4734
Name: LoanRange, dtype: int64

Creating two columns from the Loanrange an upper and lower range column

In [8]:
lower_bounds = {'a $5-10 million': 5000000,
               'b $2-5 million': 2000000,
               'c $1-2 million': 1000000,
               'd $350,000-1 million': 350000,
               'e $150,000-350,000': 150000}

upper_bounds = {'a $5-10 million': 10000000,
               'b $2-5 million': 5000000,
               'c $1-2 million': 2000000,
               'd $350,000-1 million': 1000000,
               'e $150,000-350,000': 350000}

In [9]:
df.head()

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender,CD
0,"d $350,000-1 million",AERO BOX LLC,NaN,NaN,NaN,NaN,484210.0,NaN,Unanswered,Unanswered,Unanswered,NaN,NaN,2020-05-03,The Huntington National Bank,NaN
1,"d $350,000-1 million",BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,NaN,98112.0,NaN,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,75.0,2020-05-03,"Bank of America, National Association",WA-07
2,"d $350,000-1 million",KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,NaN,92407.0,236115.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,21.0,2020-05-03,"Bank of America, National Association",CA-31
3,"d $350,000-1 million","PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,NaN,29420.0,561730.0,Sole Proprietorship,Unanswered,Male Owned,Non-Veteran,NaN,73.0,2020-05-03,Synovus Bank,SC-01
4,"d $350,000-1 million","SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,NaN,29150.0,325510.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,62.0,2020-05-03,Synovus Bank,SC-05


In [3]:
sns.countplot(data=df, y='LoanRange', order=df['LoanRange'].value_counts().index, color='darkblue')

NameError: name 'df' is not defined

In [ ]:
gb = df.groupby('LoanRange').sum()

In [ ]:
gb.reset_index(inplace=True)
#Interestingly, the total number of jobs reported is highest for the 2nd-lower loans category. It actually decreases for the higher loan amounts. Potentially these higher loan amounts have more efficient companies.

In [ ]:
sns.barplot(data=gb,
            x='JobsReported',
            y='LoanRange',
            color='darkblue',
            order=gb.sort_values(by='JobsReported', ascending=False)['LoanRange'])

#Number of jobs scales with loan amount, which makes sense. There are some outliers on the large side for the smaller loan amounts though.

In [ ]:
sns.boxenplot(data=df, x='JobsReported', y='LoanRange', order=df['LoanRange'].value_counts().index, color='white')

In [ ]:
gb_state = df.groupby('State').sum()
gb_state.reset_index(inplace=True)
#We can see a vast majority of loans are in a handful of states, with CA on top.

In [ ]:
px.choropleth(gb_state, locations='State', locationmode='USA-states', scope='usa', color='LoanLowerBound')
gb_state.sort_values(by='LoanLowerBound', ascending=False).iloc[:5]

In [ ]:
gb_state['LoanLowerBound_billions'] = gb_state['LoanLowerBound'] / 1000000000

In [ ]:
sns.barplot(data=gb_state,
            y='State',
            x='LoanLowerBound_billions',
            color='darkblue',
            order=gb_state.sort_values(by='LoanLowerBound', ascending=False).iloc[:5]['State'])
plt.xlabel('Total loans (billions)')


In [ ]:
df['JobsReported'].median()
#The distribution is very asymmetric

In [ ]:
sns.histplot(data=df[df['JobsReported'] != 0], x='JobsReported')
plt.xlabel('Jobs Reported')

In [ ]:
df.columns
#We can see JP Morgan is the top lender, with a few other big banks in the top 5. Truist Bank is also in there, which warrants more investigation.

In [ ]:
sns.countplot(data=df, y='Lender', color='darkblue', order=df['Lender'].value_counts().index[:5])

In [ ]:
gb_date = df.groupby(['DateApproved', 'BusinessType']).count().groupby(level=-1).cumsum()
gb_date.reset_index(inplace=True)

In [ ]:
gb_date.head()

In [ ]:
# since it is a cumulative sum of counts, we can use most any column for y
sns.lineplot(data=gb_date, x='DateApproved', y='LoanRange', hue='BusinessType')

In [ ]:
sns.countplot(data=df, y='BusinessType', order=df['BusinessType'].value_Lcounts().index)

In [ ]:
# too many business types on there, let's break it down -- looks like top 4 should be good
top_4_business_types = df['BusinessType'].value_counts().index[:4]

In [ ]:
from labellines import labelLines

f = plt.figure()
ax = f.gca()
for biz in top_4_business_types:
    b_df = gb_date[gb_date['BusinessType'] == biz]
    ax.plot(b_df['DateApproved'], b_df['LoanRange'], label=biz)

# Adding extra arguments 
labelLines(ax.get_lines(), xvals=[pd.to_datetime('7-1-2020')] * 4, align=False)

f.autofmt_xdate()  
plt.xlabel('Date')

As we can see, most loans were approved quickly, especially for corporations. Non-profits were the slowest to recieve loans. LLC's were next fastest after corporations.

In [ ]:
sns.color_palette('colorblind')[:4]

In [ ]:
<AxesSubplot:xlabel='count', ylabel='LoanRange'>

Above, we can see that Corporations made up the bulk of the loans in all categories, but especially the smaller loans here. Intermediate loans were a little more equal, but the biggest loans of 5-10M were dominated by corporations.